# Tensorboard 可视化

对大部分人而言，深度神经网络就像一个黑盒子，其内部的组织、结构、以及其训练过程很难理清楚，这给深度神经网络原理的理解和工程化带来了很大的挑战。为了解决这个问题，tensorboard应运而生。

Tensorboard是tensorflow内置的一个可视化工具，它通过将tensorflow程序输出的日志文件的信息可视化使得tensorflow程序的理解、调试和优化更加简单高效。

Tensorboard的可视化依赖于tensorflow程序运行输出的日志文件，因而tensorboard和tensorflow程序在不同的进程中运行。


学会用 Tensorflow 自带的 tensorboard 去可视化我们所建造出来的神经网络是一个很好的学习理解方式. 用最直观的流程图告诉你你的神经网络是长怎样,有助于你发现编程中间的问题和疑问.

## 简单使用

生成一个具有写权限的日志文件操作对象，将当前命名空间的计算图写进日志中 

    writer=tf.summary.FileWriter('/path/to/logs', tf.get_default_graph()) 

    writer.close()

## 启动服务

tensorboard –logdir /path/to/logs

## 使用案例

In [1]:
from __future__ import print_function
import tensorflow as tf

In [2]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    
    with tf.name_scope('layer'):
        
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs


构建计算图

In [3]:
# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
L1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(L1, 10, 1, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(ys - prediction),reduction_indices=[1])
    #loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),reduction_indices=[1]))

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)


In [4]:
!mkdir logs

子目录或文件 logs 已经存在。


In [5]:
sess = tf.Session()

writer = tf.summary.FileWriter("logs/", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

# direct to the local dir and run this in terminal:
# $ tensorboard --logdir=logs

通过使用这个工具我们可以很直观的看到整个神经网络的结构、框架。 通过tensorflow的工具大致可以看到神经网络差不多是这样子的

<img src=images/4_1_1.png>

同时我们也可以展开看每个layer中的一些具体的结构：

<img src=images/4_1_2.png>

好，通过阅读代码和图片我们大概知道了此处是有一个输入层（inputs），一个隐含层（layer），还有一个输出层（output） 现在可以看看如何进行可视化.

## 搭建图纸

首先从 Input 开始：

In [4]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 1])
ys = tf.placeholder(tf.float32, [None, 1])

对于input我们进行如下修改： 首先，可以为xs指定名称为x_in:

然后再次对ys指定名称y_in:

这里指定的名称将来会在可视化的图层inputs中显示出来

In [8]:
xs= tf.placeholder(tf.float32, [None, 1],name='x_in')
ys= tf.placeholder(tf.float32, [None, 1],name='y_in')

使用with tf.name_scope('inputs')可以将xs和ys包含进来，形成一个大的图层，图层的名字就是with tf.name_scope()方法里的参数。

In [9]:
with tf.name_scope('inputs'):
    # define placeholder for inputs to network
    xs = tf.placeholder(tf.float32, [None, 1])
    ys = tf.placeholder(tf.float32, [None, 1])

接下来开始编辑layer ， 请看编辑前的程序片段 ：

这里的名字应该叫layer, 下面是编辑后的:

在定义完大的框架layer之后，同时也需要定义每一个’框架‘里面的小部件：(Weights biases 和 activation function): 现在现对 Weights 定义： 定义的方法同上，可以使用tf.name.scope()方法，同时也可以在Weights中指定名称W。 即为：

接着继续定义biases ， 定义方式同上。

activation_function 的话，可以暂时忽略。因为当你自己选择用 tensorflow 中的激励函数（activation function）的时候，tensorflow会默认添加名称。 最终，layer形式如下：

In [18]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs

效果如下：（有没有看见刚才定义layer里面的“内部构件”呢？）

<img src=images/4_1_4.png>

最后编辑loss部分：将with tf.name_scope()添加在loss上方，并为它起名为loss

In [21]:
# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

这句话就是“绘制” loss了， 如下：

<img src=images/4_1_5.png>

使用with tf.name_scope()再次对train_step部分进行编辑,如下：

In [23]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

我们需要使用 tf.summary.FileWriter() (tf.train.SummaryWriter() 这种方式已经在 tf >= 0.12 版本中摒弃) 将上面‘绘画’出的图保存到一个目录中，以方便后期在浏览器中可以浏览。 

这个方法中的第二个参数需要使用sess.graph ， 因此我们需要把这句话放在获取session的后面。 这里的graph是将前面定义的框架信息收集起来，然后放在logs/目录下面。

In [24]:
sess = tf.Session() # get session
# tf.train.SummaryWriter soon be deprecated, use following
writer = tf.summary.FileWriter("logs/", sess.graph)

最后在你的terminal（终端）中 ，使用以下命令

tensorboard --logdir logs

同时将终端中输出的网址复制到浏览器中，便可以看到之前定义的视图框架了。


TensorBoard 1.8.0 at http://DESKTOP-LIHK6TS:6006 (Press CTRL+C to quit)

tensorboard 还有很多其他的参数，希望大家可以多多了解, 可以使用 tensorboard --help 查看tensorboard的详细参数

## 可能会遇到的问题 

(1) 而且与 tensorboard 兼容的浏览器是 “Google Chrome”. 使用其他的浏览器不保证所有内容都能正常显示.

(2) 同时注意, 如果使用 http://0.0.0.0:6006 网址打不开的朋友们, 请使用 http://localhost:6006, 大多数朋友都是这个问题.

(3) 请确保你的 tensorboard 指令是在你的 logs 文件根目录执行的. 如果在其他目录下, 比如 Desktop 等, 可能不会成功看到图. 比如在下面这个目录, 你要 cd 到 project 这个地方执行 /project > tensorboard --logdir logs

## Tensorboard 使用流程

TensorBoard 通过读取 TensorFlow 的事件文件来运行。TensorFlow 的事件文件包括了你会在 TensorFlow 运行中涉及到的主要数据。

下面是 TensorBoard 中汇总数据（Summary data）的大体生命周期。

[TensorBoard 简介及使用流程](https://blog.csdn.net/gsww404/article/details/78605784)


[详解TensorBoard如何调参](https://blog.csdn.net/qunnie_yi/article/details/80129434)

[Tensorboard 详解（上篇）](https://blog.csdn.net/fendouaini/article/details/80344591)

[Tensorboard 详解（下篇）](https://blog.csdn.net/fendouaini/article/details/80368770)

## 总结

根据上述内容描述，tensorboard的启动过程可以概括为以下几步：

1.创建writer，写日志文件 

    writer=tf.summary.FileWriter('/path/to/logs', tf.get_default_graph()) 

2.保存日志文件 

    writer.close() 

3.运行可视化命令，启动服务 

    tensorboard –logdir /path/to/logs 

4.打开可视化界面

    通过浏览器打开服务器访问端口http://xxx.xxx.xxx.xxx:6006

注意：tensorboard兼容Google浏览器或Firefox浏览器
